# Create Contours and Translations using Optical Flow

In this notebook, contour points and translation for each contour point are computed using Optical Flow based on this tutorial: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html.

## Imports

In [21]:
import os
import  re

import cv2
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Paths & Constants

In [22]:
ANNOTATIONS_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/480p/"
CONTOURS_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/480p/"
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'
CLOSING_KERNEL_SIZE = 25

In [23]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

## Functions

In [24]:
def load_gray_img(img_path):
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img_gray

In [25]:
def close_image(image, closing_kernel_size):
    '''Returns the image that is closed with a elliptical kernel.'''
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(closing_kernel_size, closing_kernel_size))
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    return closing

In [26]:
def extract_longest_contour(image, closing_kernel_size):
    '''Returns the contour with the most points from a given image.'''
    
    # Close image
    image_closed = close_image(image, closing_kernel_size)
    
    # Apply threshold to turn it into binary image
    ret, thresh = cv2.threshold(image_closed, 127, 255, 0)

    # Find contour
    # Change method for different number of points:
    # CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, CHAIN_APPROX_TC89_L1, CHAIN_APPROX_TC89_KCOS
    contours, _ = cv2.findContours(image=thresh,
                                   mode=cv2.RETR_TREE,
                                   method=cv2.CHAIN_APPROX_TC89_KCOS)
    
    # Get longest contour from contours
    longest_contour = max(contours, key=len).astype(np.float32)
    
    return longest_contour

In [27]:
def get_translations(contour_0, contour_1):
    '''Returns the translations for each point in contour_0 to contour_1.'''
    
    contour_0 = np.squeeze(contour_0)
    contour_1 = np.squeeze(contour_1)
    
    translations = np.empty([0, 2], int)
    
    for contour_point_0, contour_point_1 in zip(contour_0, contour_1):
        translation = np.subtract(contour_point_1, contour_point_0)
        translations = np.append(translations, translation.reshape([1,2]), axis=0)
    
    return translations

In [28]:
def save_annotation_with_contour_and_translation(annotation, contour, translation, path):
    '''Saves annotation plotted with contour and translation to a given path.'''
    
    plt.figure(figsize=(15,10))
    
    # Plot contour
    contour = np.squeeze(contour)
    plt.scatter(contour[:, 0], contour[:, 1])
    
    # Plot annotation
    plt.imshow(annotation)
    
    # Plot translation
    for c, t in zip(contour, translation):
        plt.arrow(c[0], c[1],
                  t[0], t[1],
                  width=1, color='r')  
    
    # Save image
    plt.savefig(path, bbox_inches=0)    
    plt.close()

In [29]:
def create_contours_and_translations(annotations_folders_path,
                                     contours_folders_path,
                                     translations_folders_path,
                                     closing_kernel_size):

    # Get list of sequences
    sequences = os.listdir(annotations_folders_path)
    sequences.sort()
    
    # Iterate through sequences
    for i, sequence in enumerate(sequences):
        
        # Debug
        #if (i > 0): break
            
        print('#{}: {}'.format(i, sequence))
        
        # Create folder to save Contours
        contours_folder_path = os.path.join(contours_folders_path, sequence)
        if not os.path.exists(contours_folder_path):
            os.makedirs(contours_folder_path)        
        
        # Create folder to save Translations
        translations_folder_path = os.path.join(translations_folders_path, sequence)
        if not os.path.exists(translations_folder_path):
            os.makedirs(translations_folder_path)
        
        # Get list of frames
        frames = os.listdir(os.path.join(annotations_folders_path, sequence))
        if '.ipynb_checkpoints' in frames:
            frames.remove('.ipynb_checkpoints')
        frames.sort()
        
        # Iterate through frames
        for j, frame in enumerate(frames):
            
            # Debug
            #if (j > 0): break
            #print('\t#{}: {}'.format(j, frame))
                        
            # Get path to frames
            frame_0_path = os.path.join(annotations_folders_path, sequence, frame)
            try:
                frame_1_path = os.path.join(annotations_folders_path, sequence, frames[j+1])
            # Break if frame_0 is last frame
            except IndexError as e:
                break
            
            # Load frames as gray img
            frame_0_gray = load_gray_img(frame_0_path)
            frame_1_gray = load_gray_img(frame_1_path)
            
            # Extract longest contour and save it
            contour_0 = extract_longest_contour(frame_0_gray, closing_kernel_size)
            np.save(os.path.join(contours_folder_path, frame[:5]), contour_0)
            
            # calculate optical flow to get contour_1
            contour_1, st, err = cv2.calcOpticalFlowPyrLK(frame_0_gray, frame_1_gray, 
                                                          contour_0, None, **lk_params)
            
            # Get translation and save it
            translation_0_1 = get_translations(contour_0, contour_1)
            np.save(os.path.join(translations_folder_path, frame[:5]), translation_0_1)
            
            # Save annotation with contour and translation
            #annotation = cv2.imread(os.path.join(annotations_folders_path, sequence, frame))
            #save_annotation_with_contour_and_translation(annotation, contour_0, translation_0_1, 
            #    os.path.join(translations_folder_path, frame[:5] + '.png'))            

In [30]:
create_contours_and_translations(ANNOTATIONS_FOLDERS_PATH,
                                 CONTOURS_FOLDERS_PATH,
                                 TRANSLATIONS_FOLDERS_PATH,
                                 CLOSING_KERNEL_SIZE)

#0: bear
#1: blackswan
#2: bmx-bumps
#3: bmx-trees
#4: boat
#5: breakdance
#6: breakdance-flare
#7: bus
#8: camel
#9: car-roundabout
#10: car-shadow
#11: car-turn
#12: cows
#13: dance-jump
#14: dance-twirl
#15: dog
#16: dog-agility
#17: drift-chicane
#18: drift-straight
#19: drift-turn
#20: elephant
#21: flamingo
#22: goat
#23: hike
#24: hockey
#25: horsejump-high
#26: horsejump-low
#27: kite-surf
#28: kite-walk
#29: libby
#30: lucia
#31: mallard-fly
#32: mallard-water
#33: motocross-bumps
#34: motocross-jump
#35: motorbike
#36: paragliding
#37: paragliding-launch
#38: parkour
#39: rhino
#40: rollerblade
#41: scooter-black
#42: scooter-gray
#43: soapbox
#44: soccerball
#45: stroller
#46: surf
#47: swing
#48: tennis
#49: train
